In [1]:
import pandas as pd
import gspread
from fuzzywuzzy import fuzz, process

In [2]:
# service account json credentials 
SERVICE_ACCOUNT_FILE = 'accesspysheet-bd152702637a.json'
# Authenticate with the Google Sheets API
gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)

In [3]:
# Open the Google Sheet by URL
SHEET_URL = "https://docs.google.com/spreadsheets/d/1h4HlBY1_vOAuFmWQxTgHRr3075wgfxfroIWmHZm4b98/edit?gid=0#gid=0"
sh = gc.open_by_url(SHEET_URL)

In [4]:

# Select the first worksheet
worksheet = sh.get_worksheet(0)

# Extract all data as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# Drop duplicates based on the first column Submission ID
df = df.drop_duplicates(subset=df.columns[0])

# Ensure the 'working title' column is treated as strings
df['What is your working project title?'] = df['What is your working project title?'].astype(str)

# Convert the 'Submitted at' column to datetime
df['Submitted at'] = pd.to_datetime(df['Submitted at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Merge columns from indices 11 and 14 into a new column called 'CATEGORY'
df['STUDY_TYPE'] = df.iloc[:, [11, 14]].apply(lambda x: ' '.join(x), axis=1)


In [5]:
# Function to check fuzzy matches for duplicates in the 3rd column
def find_fuzzy_duplicates(df, column_index, threshold=90):
    duplicate_rows = []
    for i, value1 in enumerate(df.iloc[:, column_index]):
        for j, value2 in enumerate(df.iloc[:, column_index]):
            if i < j and fuzz.ratio(value1, value2) >= threshold:
                duplicate_rows.append(i)
                duplicate_rows.append(j)
    return df.iloc[duplicate_rows].drop_duplicates()

In [6]:
# Find fuzzy duplicates in the 'working title' column
duplicate_pairs = find_fuzzy_duplicates(df, 3, threshold=90)
duplicate_pairs

,Submission ID,Respondent ID,Submitted at,What is your working project title?,Project Leader,Other Project Members?,What is your email address?,What is your best contact number?,Which Department do you belong to? (Please choose ONE),What is your staffing role? (Please choose ONE),...,Protocol Required Elements (Check here to view document and download),New Study Submission Checklist (optional info),New Study Submission Checklist (optional info) (Check here to view document and download),Request Preview to PHI for Trinity Health Network on EPIC (good for data scrubbing prior to IRB approval),Request Preview to PHI for Trinity Health Network on EPIC (good for data scrubbing prior to IRB approval) (Check here to view document and download),Will you submit this project for the upcoming SAMC's Annual Research Poster Day?,Published or submitting for peer reviewed publication?,"Please submit supporting documents (PDF, DOC, PPT) that you are reporting","For verification purposes, please sign below using mouse or touch (mobile)",STUDY_TYPE


In [7]:
column_indices = {index: title for index, title in enumerate(df.columns)}
column_indices

{0: 'Submission ID',
 1: 'Respondent ID',
 2: 'Submitted at',
 3: 'What is your working project title?',
 4: 'Project Leader',
 5: 'Other Project Members?',
 6: 'What is your email address?',
 7: 'What is your best contact number?',
 8: 'Which Department do you belong to? (Please choose ONE)',
 9: 'What is your staffing role? (Please choose ONE)',
 10: 'Is this a NEW scholarly activity project that you are submitting performed under SAMC GME?',
 11: 'Please select the category of scholarly activity that you wish to report performed under SAMC GME?',
 12: 'Was this or in the process of submission to IRB ?',
 13: 'When was this project started/completed?',
 14: 'Is this project a Quality Improvement (QI), Research, or Case Report (Please choose ONE) ',
 15: 'How does QI differ from research?',
 16: 'How does QI differ from research? (Check here to view document and download)',
 17: 'CATEGORY',
 18: 'EMAIL CASE TITLE',
 19: 'EMAIL CASE LINK',
 20: 'EMAIL IRB TITLE',
 21: 'EMAIL IRB 1 TXT'

In [8]:
# Select the specified columns by their indices
selected_columns = [3, 4, 8, 9, 84, 88]
new_df = df.iloc[:, selected_columns]
column_renames = ['Title', 'PI', 'Dept', 'Role', 'RD_Submit', 'Category']

# Rename the columns using the column_renames list
new_df.columns = column_renames
# Display the new dataframe
new_df.rename(columns=dict(zip(new_df.columns, column_renames)), inplace=True)
new_df

/var/folders/29/xd9b3r4958l8nyrjxk98cv5c0000gn/T/ipykernel_59860/1223209008.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns=dict(zip(new_df.columns, column_renames)), inplace=True)


,Title,PI,Dept,Role,RD_Submit,Category
0,"3D Print, Design and Development of a Low-Cost...",Lea Ann Urita,Emergency Medicine,PGY1,YES,Research
1,Chocolate or blood - A Case of Pesticide Induc...,Ruqayya Gill,Emergency Medicine,PGY3,YES,Case Report
2,Is patient satisfaction affected by mode of ar...,Ruqayya Gill,Emergency Medicine,PGY2,YES,Research
3,"Pneumosepsis, hypoxia, anemia, and hemoptysis ...",Martin Young,Emergency Medicine,PGY3,YES,Case Report
4,Rare case of ovarian torsion and appendicitis ...,Gullerana Ahmad,Emergency Medicine,PGY3,YES,Case Report
...,...,...,...,...,...,...
62,ground level fall to fractureless cord compres...,Jon Schulze,Transitional Medicine,PGY1,YES,Case Report
63,Impact of Asynchronous Resident Pre-Dictation ...,Christian Lee,Transitional Medicine,PGY1,YES,Research
64,Quantitative susceptibility mapping (QSM) corr...,"Kevin Lee, MD",Transitional Medicine,PGY1,YES,Research
65,Three-in-One: Pulmonary Co-infection With a Vi...,Dr. Mohammed Elhassan,Transitional Medicine,PGY1,YES,Case Report


In [9]:
# Remove spaces and unnecessary strings in the 'Category' column
new_df['Category'] = new_df['Category'].str.replace(r'\s+', '', regex=True).str.replace(r'[^a-zA-Z]', '', regex=True)

# Display the updated dataframe
new_df

/var/folders/29/xd9b3r4958l8nyrjxk98cv5c0000gn/T/ipykernel_59860/3355316059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Category'] = new_df['Category'].str.replace(r'\s+', '', regex=True).str.replace(r'[^a-zA-Z]', '', regex=True)


,Title,PI,Dept,Role,RD_Submit,Category
0,"3D Print, Design and Development of a Low-Cost...",Lea Ann Urita,Emergency Medicine,PGY1,YES,Research
1,Chocolate or blood - A Case of Pesticide Induc...,Ruqayya Gill,Emergency Medicine,PGY3,YES,CaseReport
2,Is patient satisfaction affected by mode of ar...,Ruqayya Gill,Emergency Medicine,PGY2,YES,Research
3,"Pneumosepsis, hypoxia, anemia, and hemoptysis ...",Martin Young,Emergency Medicine,PGY3,YES,CaseReport
4,Rare case of ovarian torsion and appendicitis ...,Gullerana Ahmad,Emergency Medicine,PGY3,YES,CaseReport
...,...,...,...,...,...,...
62,ground level fall to fractureless cord compres...,Jon Schulze,Transitional Medicine,PGY1,YES,CaseReport
63,Impact of Asynchronous Resident Pre-Dictation ...,Christian Lee,Transitional Medicine,PGY1,YES,Research
64,Quantitative susceptibility mapping (QSM) corr...,"Kevin Lee, MD",Transitional Medicine,PGY1,YES,Research
65,Three-in-One: Pulmonary Co-infection With a Vi...,Dr. Mohammed Elhassan,Transitional Medicine,PGY1,YES,CaseReport


In [10]:
new_df_yes = new_df[new_df['RD_Submit'] == 'YES']
new_df_yes

,Title,PI,Dept,Role,RD_Submit,Category
0,"3D Print, Design and Development of a Low-Cost...",Lea Ann Urita,Emergency Medicine,PGY1,YES,Research
1,Chocolate or blood - A Case of Pesticide Induc...,Ruqayya Gill,Emergency Medicine,PGY3,YES,CaseReport
2,Is patient satisfaction affected by mode of ar...,Ruqayya Gill,Emergency Medicine,PGY2,YES,Research
3,"Pneumosepsis, hypoxia, anemia, and hemoptysis ...",Martin Young,Emergency Medicine,PGY3,YES,CaseReport
4,Rare case of ovarian torsion and appendicitis ...,Gullerana Ahmad,Emergency Medicine,PGY3,YES,CaseReport
...,...,...,...,...,...,...
62,ground level fall to fractureless cord compres...,Jon Schulze,Transitional Medicine,PGY1,YES,CaseReport
63,Impact of Asynchronous Resident Pre-Dictation ...,Christian Lee,Transitional Medicine,PGY1,YES,Research
64,Quantitative susceptibility mapping (QSM) corr...,"Kevin Lee, MD",Transitional Medicine,PGY1,YES,Research
65,Three-in-One: Pulmonary Co-infection With a Vi...,Dr. Mohammed Elhassan,Transitional Medicine,PGY1,YES,CaseReport


In [11]:
pivot_table = new_df_yes.pivot_table(index='Dept', columns=['Category'], aggfunc='size', fill_value=0)
# Add a new column 'Total' to get the total count for each department
pivot_table['Total'] = pivot_table.sum(axis=1)

# Calculate the sum for each category column
category_totals = pivot_table.sum(axis=0)

# Append the totals as a new row to the pivot_table
pivot_table.loc['Total'] = category_totals

pivot_table

Category,CaseReport,QI,Research,Total
Dept,,,,
Emergency Medicine,4,0,3,7
Family Medicine,6,2,0,8
Internal Medicine,24,17,2,43
Transitional Medicine,2,0,4,6
Total,36,19,9,64


In [ ]:
from IPython.display import display, HTML

# Display the dataframe with a fixed height and enable scrolling
display(HTML('<div style="height: 400px; overflow: auto;">{}</div>'.format(new_df_yes.to_html(index=False))))

In [ ]:
# Verify the conversion
print(df['Submitted at'].dtype)  # Should now show 'datetime64[ns]'


In [13]:
from oauth2client.service_account import ServiceAccountCredentials
import json
import streamlit as st

In [14]:
gcp_secrets = st.secrets["gcp_service_account"]

# Convert TOML to JSON format for gspread
credentials_dict = {key: gcp_secrets[key] for key in gcp_secrets}
credentials_dict["private_key"] = credentials_dict["private_key"].replace("\\n", "\n")

In [15]:
# Authenticate with gspread
creds = ServiceAccountCredentials.from_json_keyfile_dict(credentials_dict, scopes=["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"])
client = gspread.authorize(creds)


In [18]:
# Open Google Sheet
sheet = client.open("SAMC Scholarly Activity Form").worksheet("Sheet1")
data = sheet.get_all_records()